In [1]:
from keras.models import load_model
from os import listdir
from scipy import misc
from skimage import io,color
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import pickle
import _pickle as cPickle
from keras.models import Sequential
from keras.layers import Dense,Embedding,LSTM,GRU

%matplotlib inline

/usr/lib/python3.4/importlib/_bootstrap.py:321: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  return f(*args, **kwds)
Using TensorFlow backend.


In [13]:
train_data=cPickle.load(open('RNN_Train_6.p','rb'))
test_data=cPickle.load(open('RNN_Test_6.p','rb'))

x_train,y_train=[train_data[i][0] for i in range(len(train_data))],[train_data[i][1] for i in range(len(train_data))]
x_test,y_test=[test_data[i][0] for i in range(len(test_data))],[test_data[i][1] for i in range(len(test_data))]

In [14]:
print(len(x_train))
print(len(x_test))
x_train=np.array(x_train)
x_test=np.array(x_test)
print(x_train.shape)
y_train=np.asarray(y_train)
y_test=np.asarray(y_test)
print(y_train.shape)

87
26
(87, 50, 1024)
(87, 9)


In [5]:
print(x_train[86].shape)
print(y_train[86])

(50, 3200)
[0. 0. 0. 0. 0. 0. 0. 0. 1.]


In [ ]:
# 15 epochs for 128 max and 20/25 max for 256
from keras.models import Sequential
from keras import optimizers
from keras.layers import Dense,Embedding,LSTM,GRU,RNN,SimpleRNN,Dropout
model = Sequential()
# adam=optimizers.Adam(lr=0.001,decay=1e-4)
# rms=optimizers.RMSprop()
rms=optimizers.RMSprop(lr=0.001,decay=1e-4)
model.add(LSTM(128,input_shape=(x_train.shape[1],x_train.shape[2]),dropout=0.2,recurrent_dropout=0.2)) #256 gave best
# model.add(LSTM(256,dropout=0.3,recurrent_dropout=0.3))
# model.add(Dense(256,activation='relu'))
# model.add(Dropout(0.5))
model.add(Dense(9, activation='softmax'))
print(model.summary())
model.compile(loss='categorical_crossentropy', optimizer=rms, metrics=['accuracy'])
model.fit(x_train,y_train,epochs=20,batch_size=4,validation_data=(x_test,y_test),shuffle=True)

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_5 (LSTM)                (None, 128)               590336    
_________________________________________________________________
dense_6 (Dense)              (None, 9)                 1161      
Total params: 591,497
Trainable params: 591,497
Non-trainable params: 0
_________________________________________________________________
None
Train on 87 samples, validate on 26 samples
Epoch 1/20
87/87 [==============================] - 4s 48ms/step - loss: 1.9106 - acc: 0.3793 - val_loss: 1.7415 - val_acc: 0.5769
Epoch 2/20
87/87 [==============================] - 2s 26ms/step - loss: 0.9433 - acc: 0.8161 - val_loss: 1.7407 - val_acc: 0.6154
Epoch 3/20
87/87 [==============================] - 2s 27ms/step - loss: 0.5601 - acc: 0.9080 - val_loss: 1.6304 - val_acc: 0.5769
Epoch 4/20
87/87 [==============================] - 2s 27ms/step - loss: 0.3644 - acc: 0.9

In [10]:
# model.save('Model_73_7.h5')

In [8]:
model=load_model('Model_69_7.h5')
# predict=model.predict(x_test)
# model.evaluate(x_test,y_test)
print(model.summary())

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_21 (LSTM)               (None, 128)               590336    
_________________________________________________________________
dense_28 (Dense)             (None, 9)                 1161      
Total params: 591,497
Trainable params: 591,497
Non-trainable params: 0
_________________________________________________________________
None


In [32]:
categories=["Golf","Kicking","Lifting","Riding","Running","SkateBoarding","Swing-Bench","Swing-SideAngle","Walking"]
for i in range(predict.shape[0]):
    pred=np.argmax(predict[i])
    true=np.argmax(y_test[i])
    if pred!=true:
        print("True: "+str(categories[true])+"  Pred: "+str(categories[pred]))

True: Golf  Pred: Running
True: Running  Pred: Walking
True: Running  Pred: Golf
True: SkateBoarding  Pred: Walking
True: SkateBoarding  Pred: Walking
True: Swing-Bench  Pred: Running
True: Walking  Pred: Golf
